In [1]:
import os
import fsspec
from itertools import chain
import pandas as pd


from etoLib.eto_functions import grepfxn, unique, lapply_brick, rastermath

In [2]:
def _read_config(configurationfile):
    '''Read configuration file and parse out the inputs line by line'''

    # Note: if you want run rcp 8.5, then all you have to do is change the rcp_source parameter from within config file
    # It only affects grepfxn(rcp_source,all_files) below

    with open(configurationfile) as f:
        data = {}
        for line in f:
            key, value = line.strip().split(' = ')
            data[key] = value
    return(data)

In [3]:
# Note that the difference between historical and future outputs in cloud are based on these 2 configuration files.
# configurationfile = 'configurationfile_referenceET_test_future.ini'
configurationfile = 'configurationfile_referenceET_test_historical.ini'

data = _read_config(configurationfile)

print(data)

{'model_files': 's3', 'data_source': 'METDATA', 'output_folder': 'in', 'elevfile': 'in/Elevation/cgiar_srtmmerge.tif', 'tiffolder': 'C:/Users/GRouze/Desktop/ReferenceET_Python/Future-MACA/Individual/geotiffs', 'ET0_method': 'PM', 'ET0_winddat': 'yes', 'ET0_crop': 'short', 'to_clip': 'True', 'model': 'MIROC5', 'northmost': '42.54', 'southmost': '38.6', 'westmost': '-76.3', 'eastmost': '-74.23', 'pad_factor': '0.50', 'rcp_source': 'historical', 'bucket_filepath_root': 'dev-et-data/out/DelawareRiverBasin/', 'bucket_filepath_branch': 'Run08262020_Firstattemptcloudpush', 'MACA_start_bucket': 'dev-et-data/in/DelawareRiverBasin/'}


In [4]:
#! cp ../00-rouze-orig-notebooks/*.ini .

In [5]:
model_files = data['model_files']
data_source = data['data_source']
output_folder = data['output_folder']
elevfile = data['elevfile']
tiffolder = data['tiffolder']
ET0_method = data['ET0_method']
ET0_winddat = data['ET0_winddat']
ET0_crop = data['ET0_crop']
to_clip = data['to_clip']
model = data['model']
northmost = float(data['northmost'])
southmost = float(data['southmost'])
westmost = float(data['westmost'])
eastmost = float(data['eastmost'])
pad_factor = float(data['pad_factor'])
rcp_source = data['rcp_source']
MACA_start_bucket = data['MACA_start_bucket']

'''This is needed to retrieve the netCDF files from the dev-et-data AWS bucket'''
# os.chdir(model_files)
fs = fsspec.filesystem(model_files, anon=False, requester_pays=True)

all_files = fs.find(MACA_start_bucket)

# This prints all of the files in dev-et-data/in/DelawareRiverBasin/ or MACA_start_bucket...a big set of outputs, so skipped
# print(all_files)

In [6]:
all_files

['dev-et-data/in/DelawareRiverBasin/.ipynb_checkpoints/junk',
 'dev-et-data/in/DelawareRiverBasin/DRB_water_mask_inland.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/.ipynb_checkpoints/junk',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950001.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950002.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950003.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950004.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950005.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950006.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950007.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950008.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950009.tif',
 'dev-et-data/in

In [7]:
# THE CODE BELOW IS PARSED FROM THE CONDIITION WHEN DEALING WITH METDATA

In [8]:
# Split models apart that are to be used for ensemble averaging
models_parsed = [x.strip() for x in model.split(',')]

# Whittle down the number of files if the folder contains both rcp 4.5 and rcp 8.5 files
# Right now, the code can only handle one model of METDATA output (8/21/2020)
rcp_all_files = [grepfxn(rcp_source,all_files)][0]

# Iterate the files by each each specified model
models_list=[]
for i in range(len(models_parsed)):
    model_files_loop = [grepfxn(models_parsed[i],rcp_all_files)][0]
    models_list.append(model_files_loop)
        
# Flatten series of lists into one list
rcp_all_files = list(chain(*models_list))

# prints all netCDF files from 1950-2100 from MACA (radiation, precipitation, wind etc.)
print(rcp_all_files)


['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1970_1974_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1975_1979_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1980_1984_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1985_1989_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/

In [9]:
rcp_all_files

['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1970_1974_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1975_1979_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1980_1984_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1985_1989_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBa

In [10]:
# Find and compile the year blocks into a list
dfis=[]
for out in rcp_all_files:
    a=out.split('_')
    dfi = a[5]+'_'+a[6]
    dfis.append(dfi)

# print(dfis)
    
# Distill the above list into unique year blocks, as there will be duplicates from multiple climate inputs
year_all=unique(dfis);print(year_all)

# For prototyping only
year_block=0
# print(year_all)
# Print the first entry in the year list
print(year_all[year_block])

['1950_1954', '1955_1959', '1960_1964', '1965_1969', '1970_1974', '1975_1979', '1980_1984', '1985_1989', '1990_1994', '1995_1999', '2000_2004', '2005_2005']
1950_1954


In [11]:
def _make_sure_folder_exists(my_folder):

    if not os.path.exists(my_folder):
        os.makedirs(my_folder)

In [12]:
def _fake_write_geotiff(data,meta,var_name,doy,year,folder):
    
    # Now we need to convert all of these arrays back to rasterio geoTIFFs
    
    #os.chdir(folder+'/'+var_name)
    
    #filename = var_name +'_' + str(year) +  str(doy).zfill(3)+'.tif'
    _make_sure_folder_exists(folder)
    filename = folder+'/'+var_name +'_' + str(year) +  str(doy).zfill(3)+'.tif'
#     with rio.open(filename, 'w', **meta) as dst:
#         dst.write(data, 1)
        
    print("Fake Write:", filename)
    return filename

In [ ]:

# loop by each block associated with the MACA netCDF file naming structure
for year_block in range(0,len(year_all)):

    year_block_files = grepfxn(year_all[year_block],rcp_all_files)
    
    print(year_block_files)

    bounds=[southmost,northmost,westmost,eastmost]

    # precipitation
    rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # maximum air temperature
    rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # minimum air temperature
    rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    
    start_year=year_all[year_block][0:4]
    end_year=year_all[year_block][5:9]

    start=start_year+'-01-01'
    end=end_year+'-12-31'
    datetimes = pd.date_range(start=start,end=end)
#     i=10
    
    for i in range(0,rcp_pr[0][0].count):

        doy_loop = pd.Period(datetimes[i],freq='D').dayofyear
        year_loop = pd.Period(datetimes[i],freq='D').year

        # step 1: extract ith band from the raster stack
        # step 2: stack those ith bands together
        # step 3: do raster mean math from step 2
        pr_stack=[]

        # Purpose: create stacks of variables individually - this is like brick in R
        pr_ensemble = []
        tasmax_ensemble = []
        tasmin_ensemble = []


        # should be 1 array for each variable (mean of x ensembles for a given doy)
        # rcp_pr[0][0].read(1, masked=False).shape
        rcp_pr_doy = rastermath(rcp_pr[0], i)
        rcp_tasmax_doy = rastermath(rcp_tasmax[0], i)
        rcp_tasmin_doy = rastermath(rcp_tasmin[0], i)

        # Compute tasavg
        rcp_tasavg_doy = (rcp_tasmax_doy[0] + rcp_tasmin_doy[0])/2

        rcp_pr_doy[1]['count']=1
        rcp_tasmin_doy[1]['count']=1
        rcp_tasmax_doy[1]['count']=1

        rcp_pr_doy[1]['count']=1


        gTIFF_filename_ppt = _fake_write_geotiff(data=rcp_pr_doy[0],meta=rcp_pr_doy[1],var_name='PPT',doy=doy_loop,year=year_loop,folder=output_folder)

        gTIFF_filename_tasavg = _fake_write_geotiff(data=rcp_tasavg_doy,meta=rcp_tasmax_doy[1],var_name='Temp',doy=doy_loop,year=year_loop,folder=output_folder)
    
        print("Create tif Outputs", gTIFF_filename_ppt, gTIFF_filename_tasavg)

        '''Push newly created geoTIFF into specified bucket and its filepath'''

#         os.chdir('.')
#         s3_push_delete_local(output_folder+'/' + 'PPT' + '/' + gTIFF_filename_ppt , 'dev-et-data', 
#                              'inTest/DelawareRiverBasin/PPT/'+ str(year_loop)  + '/' + gTIFF_filename_ppt )

#         s3_push_delete_local(output_folder+'/' + 'Temp' + '/' + gTIFF_filename_tasavg , 'dev-et-data', 
#                              'inTest/DelawareRiverBasin/Temp/'+ str(year_loop)  + '/' + gTIFF_filename_tasavg )

['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/RAD/Climatology/macav2metdata_rsds_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/RH/RHSMAX/Climatology/macav2metdata_rhsmax_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/RH/RHSMIN/Climatology/macav2metdata_rhsmin_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/Temp/Climatology/Tmax/macav2metdata_tasmax_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/Temp/Climatology/Tmin/macav2metdata_tasmin_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/WIND/UAS/Climatology/macav2metdata_uas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/WIND/VAS/Climatology/macav2metdata_vas_MIROC5_r1i1p1_historical_1950_1954_CO

Fake Write: in/PPT_1950061.tif
Fake Write: in/Temp_1950061.tif
Create tif Outputs in/PPT_1950061.tif in/Temp_1950061.tif
Fake Write: in/PPT_1950062.tif
Fake Write: in/Temp_1950062.tif
Create tif Outputs in/PPT_1950062.tif in/Temp_1950062.tif
Fake Write: in/PPT_1950063.tif
Fake Write: in/Temp_1950063.tif
Create tif Outputs in/PPT_1950063.tif in/Temp_1950063.tif
Fake Write: in/PPT_1950064.tif
Fake Write: in/Temp_1950064.tif
Create tif Outputs in/PPT_1950064.tif in/Temp_1950064.tif
Fake Write: in/PPT_1950065.tif
Fake Write: in/Temp_1950065.tif
Create tif Outputs in/PPT_1950065.tif in/Temp_1950065.tif
Fake Write: in/PPT_1950066.tif
Fake Write: in/Temp_1950066.tif
Create tif Outputs in/PPT_1950066.tif in/Temp_1950066.tif
Fake Write: in/PPT_1950067.tif
Fake Write: in/Temp_1950067.tif
Create tif Outputs in/PPT_1950067.tif in/Temp_1950067.tif
Fake Write: in/PPT_1950068.tif
Fake Write: in/Temp_1950068.tif
Create tif Outputs in/PPT_1950068.tif in/Temp_1950068.tif
Fake Write: in/PPT_1950069.tif
F

Fake Write: in/PPT_1950129.tif
Fake Write: in/Temp_1950129.tif
Create tif Outputs in/PPT_1950129.tif in/Temp_1950129.tif
Fake Write: in/PPT_1950130.tif
Fake Write: in/Temp_1950130.tif
Create tif Outputs in/PPT_1950130.tif in/Temp_1950130.tif
Fake Write: in/PPT_1950131.tif
Fake Write: in/Temp_1950131.tif
Create tif Outputs in/PPT_1950131.tif in/Temp_1950131.tif
Fake Write: in/PPT_1950132.tif
Fake Write: in/Temp_1950132.tif
Create tif Outputs in/PPT_1950132.tif in/Temp_1950132.tif
Fake Write: in/PPT_1950133.tif
Fake Write: in/Temp_1950133.tif
Create tif Outputs in/PPT_1950133.tif in/Temp_1950133.tif
Fake Write: in/PPT_1950134.tif
Fake Write: in/Temp_1950134.tif
Create tif Outputs in/PPT_1950134.tif in/Temp_1950134.tif
Fake Write: in/PPT_1950135.tif
Fake Write: in/Temp_1950135.tif
Create tif Outputs in/PPT_1950135.tif in/Temp_1950135.tif
Fake Write: in/PPT_1950136.tif
Fake Write: in/Temp_1950136.tif
Create tif Outputs in/PPT_1950136.tif in/Temp_1950136.tif
Fake Write: in/PPT_1950137.tif
F

Fake Write: in/PPT_1950198.tif
Fake Write: in/Temp_1950198.tif
Create tif Outputs in/PPT_1950198.tif in/Temp_1950198.tif
Fake Write: in/PPT_1950199.tif
Fake Write: in/Temp_1950199.tif
Create tif Outputs in/PPT_1950199.tif in/Temp_1950199.tif
Fake Write: in/PPT_1950200.tif
Fake Write: in/Temp_1950200.tif
Create tif Outputs in/PPT_1950200.tif in/Temp_1950200.tif
Fake Write: in/PPT_1950201.tif
Fake Write: in/Temp_1950201.tif
Create tif Outputs in/PPT_1950201.tif in/Temp_1950201.tif
Fake Write: in/PPT_1950202.tif
Fake Write: in/Temp_1950202.tif
Create tif Outputs in/PPT_1950202.tif in/Temp_1950202.tif
Fake Write: in/PPT_1950203.tif
Fake Write: in/Temp_1950203.tif
Create tif Outputs in/PPT_1950203.tif in/Temp_1950203.tif
Fake Write: in/PPT_1950204.tif
Fake Write: in/Temp_1950204.tif
Create tif Outputs in/PPT_1950204.tif in/Temp_1950204.tif
Fake Write: in/PPT_1950205.tif
Fake Write: in/Temp_1950205.tif
Create tif Outputs in/PPT_1950205.tif in/Temp_1950205.tif
Fake Write: in/PPT_1950206.tif
F

Fake Write: in/PPT_1950268.tif
Fake Write: in/Temp_1950268.tif
Create tif Outputs in/PPT_1950268.tif in/Temp_1950268.tif
Fake Write: in/PPT_1950269.tif
Fake Write: in/Temp_1950269.tif
Create tif Outputs in/PPT_1950269.tif in/Temp_1950269.tif
Fake Write: in/PPT_1950270.tif
Fake Write: in/Temp_1950270.tif
Create tif Outputs in/PPT_1950270.tif in/Temp_1950270.tif
Fake Write: in/PPT_1950271.tif
Fake Write: in/Temp_1950271.tif
Create tif Outputs in/PPT_1950271.tif in/Temp_1950271.tif
Fake Write: in/PPT_1950272.tif
Fake Write: in/Temp_1950272.tif
Create tif Outputs in/PPT_1950272.tif in/Temp_1950272.tif
Fake Write: in/PPT_1950273.tif
Fake Write: in/Temp_1950273.tif
Create tif Outputs in/PPT_1950273.tif in/Temp_1950273.tif
Fake Write: in/PPT_1950274.tif
Fake Write: in/Temp_1950274.tif
Create tif Outputs in/PPT_1950274.tif in/Temp_1950274.tif
Fake Write: in/PPT_1950275.tif
Fake Write: in/Temp_1950275.tif
Create tif Outputs in/PPT_1950275.tif in/Temp_1950275.tif
Fake Write: in/PPT_1950276.tif
F

...

Fake Write: in/PPT_1974047.tif
Fake Write: in/Temp_1974047.tif
Create tif Outputs in/PPT_1974047.tif in/Temp_1974047.tif
Fake Write: in/PPT_1974048.tif
Fake Write: in/Temp_1974048.tif
Create tif Outputs in/PPT_1974048.tif in/Temp_1974048.tif
Fake Write: in/PPT_1974049.tif
Fake Write: in/Temp_1974049.tif
Create tif Outputs in/PPT_1974049.tif in/Temp_1974049.tif
Fake Write: in/PPT_1974050.tif
Fake Write: in/Temp_1974050.tif
Create tif Outputs in/PPT_1974050.tif in/Temp_1974050.tif
Fake Write: in/PPT_1974051.tif
Fake Write: in/Temp_1974051.tif
Create tif Outputs in/PPT_1974051.tif in/Temp_1974051.tif
Fake Write: in/PPT_1974052.tif
Fake Write: in/Temp_1974052.tif
Create tif Outputs in/PPT_1974052.tif in/Temp_1974052.tif
Fake Write: in/PPT_1974053.tif
Fake Write: in/Temp_1974053.tif
Create tif Outputs in/PPT_1974053.tif in/Temp_1974053.tif
Fake Write: in/PPT_1974054.tif
Fake Write: in/Temp_1974054.tif
Create tif Outputs in/PPT_1974054.tif in/Temp_1974054.tif
Fake Write: in/PPT_1974055.tif
F

Fake Write: in/PPT_1974115.tif
Fake Write: in/Temp_1974115.tif
Create tif Outputs in/PPT_1974115.tif in/Temp_1974115.tif
Fake Write: in/PPT_1974116.tif
Fake Write: in/Temp_1974116.tif
Create tif Outputs in/PPT_1974116.tif in/Temp_1974116.tif
Fake Write: in/PPT_1974117.tif
Fake Write: in/Temp_1974117.tif
Create tif Outputs in/PPT_1974117.tif in/Temp_1974117.tif
Fake Write: in/PPT_1974118.tif
Fake Write: in/Temp_1974118.tif
Create tif Outputs in/PPT_1974118.tif in/Temp_1974118.tif
Fake Write: in/PPT_1974119.tif
Fake Write: in/Temp_1974119.tif
Create tif Outputs in/PPT_1974119.tif in/Temp_1974119.tif
Fake Write: in/PPT_1974120.tif
Fake Write: in/Temp_1974120.tif
Create tif Outputs in/PPT_1974120.tif in/Temp_1974120.tif
Fake Write: in/PPT_1974121.tif
Fake Write: in/Temp_1974121.tif
Create tif Outputs in/PPT_1974121.tif in/Temp_1974121.tif
Fake Write: in/PPT_1974122.tif
Fake Write: in/Temp_1974122.tif
Create tif Outputs in/PPT_1974122.tif in/Temp_1974122.tif
Fake Write: in/PPT_1974123.tif
F

Fake Write: in/PPT_1974184.tif
Fake Write: in/Temp_1974184.tif
Create tif Outputs in/PPT_1974184.tif in/Temp_1974184.tif
Fake Write: in/PPT_1974185.tif
Fake Write: in/Temp_1974185.tif
Create tif Outputs in/PPT_1974185.tif in/Temp_1974185.tif
Fake Write: in/PPT_1974186.tif
Fake Write: in/Temp_1974186.tif
Create tif Outputs in/PPT_1974186.tif in/Temp_1974186.tif
Fake Write: in/PPT_1974187.tif
Fake Write: in/Temp_1974187.tif
Create tif Outputs in/PPT_1974187.tif in/Temp_1974187.tif
Fake Write: in/PPT_1974188.tif
Fake Write: in/Temp_1974188.tif
Create tif Outputs in/PPT_1974188.tif in/Temp_1974188.tif
Fake Write: in/PPT_1974189.tif
Fake Write: in/Temp_1974189.tif
Create tif Outputs in/PPT_1974189.tif in/Temp_1974189.tif
Fake Write: in/PPT_1974190.tif
Fake Write: in/Temp_1974190.tif
Create tif Outputs in/PPT_1974190.tif in/Temp_1974190.tif
Fake Write: in/PPT_1974191.tif
Fake Write: in/Temp_1974191.tif
Create tif Outputs in/PPT_1974191.tif in/Temp_1974191.tif
Fake Write: in/PPT_1974192.tif
F

Fake Write: in/PPT_1974254.tif
Fake Write: in/Temp_1974254.tif
Create tif Outputs in/PPT_1974254.tif in/Temp_1974254.tif
Fake Write: in/PPT_1974255.tif
Fake Write: in/Temp_1974255.tif
Create tif Outputs in/PPT_1974255.tif in/Temp_1974255.tif
Fake Write: in/PPT_1974256.tif
Fake Write: in/Temp_1974256.tif
Create tif Outputs in/PPT_1974256.tif in/Temp_1974256.tif
Fake Write: in/PPT_1974257.tif
Fake Write: in/Temp_1974257.tif
Create tif Outputs in/PPT_1974257.tif in/Temp_1974257.tif
Fake Write: in/PPT_1974258.tif
Fake Write: in/Temp_1974258.tif
Create tif Outputs in/PPT_1974258.tif in/Temp_1974258.tif
Fake Write: in/PPT_1974259.tif
Fake Write: in/Temp_1974259.tif
Create tif Outputs in/PPT_1974259.tif in/Temp_1974259.tif
Fake Write: in/PPT_1974260.tif
Fake Write: in/Temp_1974260.tif
Create tif Outputs in/PPT_1974260.tif in/Temp_1974260.tif
Fake Write: in/PPT_1974261.tif
Fake Write: in/Temp_1974261.tif
Create tif Outputs in/PPT_1974261.tif in/Temp_1974261.tif
Fake Write: in/PPT_1974262.tif
F

Fake Write: in/PPT_1974326.tif
Fake Write: in/Temp_1974326.tif
Create tif Outputs in/PPT_1974326.tif in/Temp_1974326.tif
Fake Write: in/PPT_1974327.tif
Fake Write: in/Temp_1974327.tif
Create tif Outputs in/PPT_1974327.tif in/Temp_1974327.tif
Fake Write: in/PPT_1974328.tif
Fake Write: in/Temp_1974328.tif
Create tif Outputs in/PPT_1974328.tif in/Temp_1974328.tif
Fake Write: in/PPT_1974329.tif
Fake Write: in/Temp_1974329.tif
Create tif Outputs in/PPT_1974329.tif in/Temp_1974329.tif
Fake Write: in/PPT_1974330.tif
Fake Write: in/Temp_1974330.tif
Create tif Outputs in/PPT_1974330.tif in/Temp_1974330.tif
Fake Write: in/PPT_1974331.tif
Fake Write: in/Temp_1974331.tif
Create tif Outputs in/PPT_1974331.tif in/Temp_1974331.tif
Fake Write: in/PPT_1974332.tif
Fake Write: in/Temp_1974332.tif
Create tif Outputs in/PPT_1974332.tif in/Temp_1974332.tif
Fake Write: in/PPT_1974333.tif
Fake Write: in/Temp_1974333.tif
Create tif Outputs in/PPT_1974333.tif in/Temp_1974333.tif
Fake Write: in/PPT_1974334.tif
F

In [ ]:
! ls in/temp

In [ ]:
! rio info in/temp/2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.tif

In [ ]:
! rio info in/temp/data_tasmax_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.tif